## In questo file andiamo ad individuare le 5 feature più importanti per fare clustering

In [16]:
# Importiamo le librerie

from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import combinations
from os import listdir
from os.path import isfile, join
import numpy as np
import os

In [17]:
path_file = os.path.abspath(os.getcwd()) #prendiamo il path in cui si trova il file su cui stiamo lavorando
print(path_file)
os.chdir(path_file) #cambiamo directory al fine di poter prendere i file csv per l'analisi della selezione delle features
data = r"/top_feature"

comb = list(combinations(range(0,5), 2))

/home/spalazzi/Downloads/AdvancedCyberSecurity-main


In [18]:
final_path = path_file + data
print(final_path)

/home/spalazzi/Downloads/AdvancedCyberSecurity-main/top_feature


In [19]:
anova = pd.DataFrame() #creazione di un dataframe vuoto
extra_tree = pd.DataFrame() #creazione di un dataframe vuoto
rfe = pd.DataFrame() #creazione di un dataframe vuoto
svm = pd.DataFrame() #creazione di un dataframe vuoto

In [20]:

for cart_corrente, sottocartelle, files in os.walk(final_path):
    for cartella in sottocartelle:
        results = os.path.join(final_path,cartella)
        for cartella_corrente, sottocart, file in os.walk(results):
            for csv_file in file:
                file_corrente = os.path.join(results,csv_file)
                df_csv = pd.read_csv(file_corrente)
                if file_corrente.__contains__("anova.csv"):
                    anova = pd.concat([anova,df_csv],ignore_index=True)
                elif file_corrente.__contains__("ExtraTree.csv"):
                    extra_tree = pd.concat([extra_tree,df_csv],ignore_index=True)
                elif file_corrente.__contains__("RFE.csv"):
                    rfe = pd.concat([rfe,df_csv],ignore_index=True)
                elif file_corrente.__contains__("SVM.csv"):
                    svm = pd.concat([svm,df_csv],ignore_index=True)


In [21]:
# Andiamo a normalizzare tutti gli score delle feature, in maniera tale da avere valori compresi tra 0 e 1

anova['F_Score'] = anova['F_Score']/anova['F_Score'].max()
extra_tree['F_Score'] = extra_tree['F_Score']/extra_tree['F_Score'].max()
svm['F_Score'] = svm['F_Score']/svm['F_Score'].max()
rfe['F_Score'] = rfe['F_Score']/rfe['F_Score'].max()

In [22]:
# Facciamo una concatenazione dei vari dataframe.
# Saranno utilizzati per prendere le feature che si ripetono di più e che hanno i punteggi maggiori

total_feature = pd.concat([anova,extra_tree,svm,rfe],ignore_index=True)
total_feature_count = pd.concat([anova,extra_tree,svm,rfe],ignore_index=True)

In [23]:
# Andiamo a eseguire un operazione di group by, con successivo count delle occorrenze dei vari campi

total_feature_count = total_feature_count.groupby(['Input_Features']).count().reset_index().sort_values(by='F_Score',ascending=False)

# Andiamo a rinominare la colonna F_Score, che adesso contiene i valori di count

total_feature_count.rename(columns = {'F_Score':'Conto'}, inplace = True)
print(total_feature_count)

       Input_Features  Conto
47  Init Fwd Win Byts     23
40   Fwd Seg Size Min     19
46  Init Bwd Win Byts     17
25        Flow Pkts/s     14
34    Fwd Pkt Len Max     13
..                ...    ...
56       SYN Flag Cnt      1
36    Fwd Pkt Len Min      1
15     CWE Flag Count      1
41      Fwd URG Flags      1
33      Fwd PSH Flags      1

[66 rows x 2 columns]


In [24]:
# Delle delle feature più frequenti mi interessano le prime dieci

most_frequent = total_feature_count[:10]

print(most_frequent)

       Input_Features  Conto
47  Init Fwd Win Byts     23
40   Fwd Seg Size Min     19
46  Init Bwd Win Byts     17
25        Flow Pkts/s     14
34    Fwd Pkt Len Max     13
48       PSH Flag Cnt     12
10    Bwd Pkt Len Max     12
37    Fwd Pkt Len Std     12
38         Fwd Pkts/s     12
39   Fwd Seg Size Avg     11


In [25]:
# Ci serve ancora il parametro F_Scores, quindi creiamo un nuovo dataframe con queste righe.
# Abbiamo le dieci righe, però, sono ancora presenti le ridondanze che prima avevamo eliminato.

test = pd.DataFrame(data=[],columns=total_feature.columns)

for a in most_frequent['Input_Features']:
    test = pd.concat([test,total_feature[total_feature['Input_Features'] == a]])

print(test)

        Input_Features   F_Score
6    Init Fwd Win Byts  0.000005
49   Init Fwd Win Byts  0.000755
74   Init Fwd Win Byts  0.007213
88   Init Fwd Win Byts  0.083067
103  Init Fwd Win Byts  0.162237
..                 ...       ...
168   Fwd Seg Size Avg  0.120999
181   Fwd Seg Size Avg  0.258833
234   Fwd Seg Size Avg  0.292542
329   Fwd Seg Size Avg  1.000000
347   Fwd Seg Size Avg  1.000000

[145 rows x 2 columns]


In [26]:
# Creo una variabile all'interno del quale vado a salvare il groupby del precedente dataframe, solo che in uesto caso
# andiamo a sommare i punteggi di F_Scores, invece di contare le occorrenze.

group_by = test.groupby(['Input_Features'])['F_Score'].sum().reset_index()

In [27]:
# Facciamo un ordinamento del dataframe precedente

print(group_by.sort_values(by='F_Score',ascending=False))

      Input_Features    F_Score
8  Init Fwd Win Byts  11.565773
7  Init Bwd Win Byts   9.631053
6   Fwd Seg Size Min   7.735106
1        Flow Pkts/s   6.271195
3    Fwd Pkt Len Std   4.895337
0    Bwd Pkt Len Max   3.993803
2    Fwd Pkt Len Max   3.638016
4         Fwd Pkts/s   3.043988
5   Fwd Seg Size Avg   2.861467
9       PSH Flag Cnt   1.489924


In [28]:
# A questo punto, terminiamo l'operazione andando a moltiplicare la frequenza per la somma dei punteggi delle varie feature.

last_feature = pd.DataFrame(data=[],columns=group_by.columns)

for a,b in zip(most_frequent['Input_Features'],most_frequent['Conto']):
    last_feature.loc[len(last_feature)] =  {'Input_Features':a,'F_Score':group_by[group_by['Input_Features'] == a]['F_Score'].values[0]*b}
    

In [29]:
# Facciamo una stampa ordinata

print(last_feature.sort_values(by='F_Score',ascending=False))

      Input_Features     F_Score
0  Init Fwd Win Byts  266.012777
2  Init Bwd Win Byts  163.727902
1   Fwd Seg Size Min  146.967007
3        Flow Pkts/s   87.796736
7    Fwd Pkt Len Std   58.744040
6    Bwd Pkt Len Max   47.925639
4    Fwd Pkt Len Max   47.294212
8         Fwd Pkts/s   36.527860
9   Fwd Seg Size Avg   31.476138
5       PSH Flag Cnt   17.879092


In [30]:
# A questo punto ci interessano solo le prime 5 feature, e tutte le loro possibili combinazioni di due elementi.

esito_finale = pd.DataFrame(data=[],columns=['Feature_1','Feature_2'])
top_index = []

# Qui andiamo a prendere gli indici delle prime 5 features

for a in range(0,5):
    top_index.append(last_feature.sort_values(by='F_Score',ascending=False).index[a])
    
# Andiamo a inserire tutte le possibili combinazioni nel dataframe

for b in comb:

    esito_finale.loc[len(esito_finale)] =  {
        'Feature_1': last_feature.loc[top_index[b[0]]]['Input_Features'],
        'Feature_2': last_feature.loc[top_index[b[1]]]['Input_Features']
        }

In [31]:
print(esito_finale)

           Feature_1          Feature_2
0  Init Fwd Win Byts  Init Bwd Win Byts
1  Init Fwd Win Byts   Fwd Seg Size Min
2  Init Fwd Win Byts        Flow Pkts/s
3  Init Fwd Win Byts    Fwd Pkt Len Std
4  Init Bwd Win Byts   Fwd Seg Size Min
5  Init Bwd Win Byts        Flow Pkts/s
6  Init Bwd Win Byts    Fwd Pkt Len Std
7   Fwd Seg Size Min        Flow Pkts/s
8   Fwd Seg Size Min    Fwd Pkt Len Std
9        Flow Pkts/s    Fwd Pkt Len Std


In [32]:
top5 = esito_finale.Feature_1.unique()
top5 = np.append(top5,esito_finale.Feature_2.unique())
top5 = pd.Series(top5)
top5 = top5.unique()
top5 = pd.Series(top5)
print(top5)

top5.to_csv('top_feature/top5_totale.csv',index=False)

0    Init Fwd Win Byts
1    Init Bwd Win Byts
2     Fwd Seg Size Min
3          Flow Pkts/s
4      Fwd Pkt Len Std
dtype: object


In [33]:
esito_finale.to_csv('top_feature/combinazioni_totale.csv',index=False,header=False)